# Feature Engineering and Encoding
**In this file, we will handle the cases in which the features values ​​are null or equal to 0. Next, some features need to be encoded to prepare the dataset for training**

In [21]:
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import numpy as np
from pyspark.sql.functions import udf
from pyspark.sql.types import BinaryType
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
import pandas as pd
from sklearn.preprocessing import LabelBinarizer

In [2]:
# Create the session
conf = SparkConf(). \
    set('spark.ui.port', "4050"). \
    set('spark.executor.memory', '15G'). \
    set('spark.driver.memory', '50G'). \
    set('spark.driver.maxResultSize', '40G'). \
    setAppName("PySparkProject"). \
    set('spark.executor.cores', "10"). \
    setMaster("local[*]")

sc = pyspark.SparkContext.getOrCreate(conf=conf)
spark = SparkSession.builder.getOrCreate()

sc._conf.getAll()

[('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.executor.memory', '15G'),
 ('spark.driver.host', 'MarcoPC.homenet.telecomitalia.it'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.memory', '50G'),
 ('spark.executor.cores', '10'),
 ('spark.app.name', 'PySparkProject'),

In [3]:
# open data.csv as pyspark dataframe
df = spark.read.csv('data.csv', header=True, inferSchema=True)

In [4]:
# the null values in the column last_valuation must be replaced with 0
df = df.fillna({'last_valuation': 0})

In [5]:
# show the first 5 rows of the dataframe with sub_position null
df.filter(df.sub_position.isNull()).show(5)

+---------+-------------------+------------+------------------+-------------------+----+---------------+------+-----------+----------+------------+--------------------+--------+-------+-----+--------------+---------+------------+--------------+-----------+------------+-------------+-------------+---------------+-----------------+--------------+---------------+---------------+-----------------+
|player_id|             date_v|market_value|              name|         date_birth| age|current_club_id|height|citizenship|  position|sub_position|     competitions_id|clubs_id|assists|goals|minutes_played|red_cards|yellow_cards|last_valuation|appearances|games_won_pl|games_draw_pl|games_lost_pl|winning_rate_pl|games_played_club|games_won_club|games_draw_club|games_lost_club|winning_rate_club|
+---------+-------------------+------------+------------------+-------------------+----+---------------+------+-----------+----------+------------+--------------------+--------+-------+-----+--------------+

In [6]:
# the null values in the column last_position must be replaced with the value in the column position
df = df.withColumn("sub_position", coalesce(col("sub_position"), col("position")))

In [7]:
# drop instances in which the column age or date_of_birth are null
df = df.dropna(subset=('age', 'date_birth'))

In [8]:
# filter the dataframe to keep only the rows in which the column height is not 0
filtered_df = df.filter(col("height") != 0)

# average height of filtered_df
average_height = filtered_df.selectExpr("avg(height) as height_average").first()["height_average"]

In [9]:
# replace the value 0 in the column height with the mean of the column 
df = df.withColumn("height", when(col("height") == 0, average_height).otherwise(col("height")))
# TODO fare in modo che dopo la virgola ci sia 0

In [ ]:
from pyspark.ml.feature import Binarizer

# Assuming df is a Spark DataFrame
binarizer = Binarizer(threshold=0.5, inputCol="current_club_id", outputCol="binarized_current_club_id")
binarized_df = binarizer.transform(df)
print(binarized_df)

The features that we have to manage are: 
- last valuation (13,45 % null --> from null to 0)
- sub position (8,19 % null --> position)
- age (0,04 % null --> delete examples)
- date_birth (0,04 % null)
- height (some values are 0 --> average height)

We transform the column competitions id and clubs id, that are array of strings in a single binary value using label binarization.

In [10]:
# Convert clubs id and competitions id from list to string
df = df.withColumn("competitions_id", split(expr("substring(competitions_id, 2, length(competitions_id)-2)"), ", "))
df = df.withColumn("clubs_id", split(expr("substring(clubs_id, 2, length(clubs_id)-2)"), ", "))

In [11]:
# convert df in pandas dataframe
df_pandas = df.toPandas()

c:\Users\marco\AppData\Local\Programs\Python\Python310\lib\site-packages\pyspark\sql\pandas\conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
c:\Users\marco\AppData\Local\Programs\Python\Python310\lib\site-packages\pyspark\sql\pandas\conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


In [12]:
# apply pd.get_dummies to the column of arrays
dummies = pd.get_dummies(df_pandas["competitions_id"].apply(pd.Series).stack()).sum(level=0)
# concatenate the dummy variables into a single string
dummies["comp_string"] = dummies.apply(lambda x: "".join(x.astype(str)), axis=1)
# join the dummies dataframe with the original dataframe
df_pandas = df_pandas.join(dummies["comp_string"])
#df_pandas = df_pandas.drop("competitions_id")

C:\Users\marco\AppData\Local\Temp\ipykernel_14064\1573311546.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  dummies = pd.get_dummies(df_pandas["competitions_id"].apply(pd.Series).stack()).sum(level=0)


In [14]:
# apply pd.get_dummies to the column of arrays
dummies = pd.get_dummies(df_pandas["clubs_id"].apply(pd.Series).stack()).sum(level=0)
# concatenate the dummy variables into a single string
dummies["club_string"] = dummies.apply(lambda x: "".join(x.astype(str)), axis=1)
# join the dummies dataframe with the original dataframe
df_pandas = df_pandas.join(dummies["club_string"])
#df_pandas = df_pandas.drop("club_str")

C:\Users\marco\AppData\Local\Temp\ipykernel_14064\1659577897.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  dummies = pd.get_dummies(df_pandas["clubs_id"].apply(pd.Series).stack()).sum(level=0)


In [16]:
df_pandas = df_pandas.drop(["competitions_id", "clubs_id"], axis=1)

prima fare label binarization e poi nn.Encoding pytorch

da encodare:
 - player_id
  - date_v in timestemp
  - current_club_id
    - citizenship
        - position
- sub_position
- comp_string
- club_str

da droppare:
    - name
    - date_birth
    - games_played_club
    - games_won_club
    - games_draw_club
    - games_lost_club
    - competitions_id
    - clubs_id

In [27]:
# divided the column player_id for 1 million
df_pandas["player_id"] = df_pandas["player_id"].apply(lambda x: x/100000)

In [39]:
# copia df_pandas in df_c
df_c = df_pandas.copy()

In [40]:
df_c = pd.concat([df_c, pd.get_dummies(df_c['current_club_id'])], axis=1)